In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
train = pd.read_csv(r"C:\Users\elonm\Desktop\timepass\train.csv")
train.head()
test = pd.read_csv(r"C:\Users\elonm\Desktop\timepass\test.csv")
test.head()
print(train.shape, test.shape)
train.head()

(8028, 3) (2035, 2)


,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [7]:
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [6]:
test.head()

,ID,question
0,8028,homeopathy autism-vaccine-antidote ??
1,8029,What topically applied to atopic dermatitis in...
2,8030,cyst treatment with Diane 35?
3,8031,I suffer from venous insufficiency (acrocyanos...
4,8032,I am pregnant for 1 month is that essential oi...


In [8]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print("Stopwords from the NLTK corpus: ", stop_words)

def text_process(text):
    #String punctuation provides all the necessary checks
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


train['question'] = train['question'].progress_apply(lambda txt: str(txt))

Stopwords from the NLTK corpus:  {'further', 'shouldn', 'most', 'before', 'no', 'didn', 'don', 'she', 've', "needn't", 'll', 'herself', 'that', 'very', "should've", 'hadn', 'be', 'yours', 'over', 'not', 'me', 'these', 'ain', 'only', 'aren', 'its', 'had', 'am', "that'll", "you'd", 'up', 'y', 'are', 'those', 'into', "mightn't", 'own', 'each', 'i', 'we', 'because', 'there', "couldn't", 'does', 'himself', 'your', 'more', 'd', 'why', 'were', 'what', "isn't", "it's", 'being', "wouldn't", 'below', 'doing', 'did', 'through', 'an', 'against', 'until', 'by', 'out', "weren't", 'm', 'was', 'yourself', 'will', 'is', 'having', 'if', 'between', 's', 'other', 'o', 'couldn', 'how', 'mustn', 'you', 'itself', 'which', 'can', 'the', 'hers', 'won', 'doesn', 'again', 'so', 'myself', 'above', 'wouldn', "you've", 'them', 'than', "shan't", "haven't", 'nor', 'my', 'all', "didn't", 'off', 'now', 'on', 'then', 'hasn', "you're", "hasn't", 'while', 'once', "you'll", 'during', "mustn't", 'do', 'after', 'with', 'when

In [9]:
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [10]:
train.to_csv(r"C:\Users\elonm\Desktop\timepass\train.csv", index=False)
train = pd.read_csv(r"C:\Users\elonm\Desktop\timepass\train.csv")
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [11]:
exclude = set(string.punctuation)

def punct_remove(text):
    text = ''.join(ch for ch in text if ch not in exclude)
    return text

train['question'] = train['question'].progress_apply(lambda s: punct_remove(s))
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [12]:
train['question'] = train['question'].progress_apply(lambda s: s.lower())
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [13]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
stemmer = PorterStemmer()

def stemming(text):
    words = word_tokenize(text)
    new_txt = []
    for word in words:
        new_txt.append(stemmer.stem(word))
    txt = ' '.join(new_txt)
    return txt

train['question'] = train['question'].progress_apply(lambda s: stemming(s))
train.head()

,ID,question,intention
0,0,hello i m wa wrong forum for my question so i ...,28
1,1,is what the motilium reliev me against nausea,31
2,2,my doctor prescrib adenyl 2nd stamp of terribl...,28
3,3,is there a form adapt to the child of 5 year o...,44
4,4,my doctor treat me to a rhino pharingit and pr...,31


In [14]:
train.groupby('intention').count()

,ID,question
intention,,
0,204,204
1,23,23
2,22,22
3,8,8
4,76,76
5,102,102
6,26,26
7,20,20
8,134,134


In [17]:
test.shape

(2035, 2)

In [15]:
from keras.utils import to_categorical

labels = to_categorical(train['intention'], num_classes=None)
labels.shape

(8028, 51)

In [129]:
labels[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [130]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['question'], labels, test_size=0.25, random_state=42)

In [197]:
tokenizer = Tokenizer(num_words= 20000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
text_sequence = pad_sequences(sequences, maxlen=30)

In [195]:
inputs = Input(shape=[30])
layers = Embedding(20000,30,input_length=30)(inputs)
layers = LSTM(15)(layers)
layers = Dropout(0.2)(layers)
layers = Dense(256)(layers)
layers = Dropout(0.2)(layers)
layers = Dense(128)(layers)
layers = Dropout(0.2)(layers)
layers = Dense(128)(layers)
layers = Dropout(0.2)(layers)
layers = Activation('relu')(layers)
layers = Dropout(0.2)(layers)
layers = Dense(51)(layers)
layers = Activation('softmax')(layers)
model = Model(inputs=inputs,outputs=layers)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 30)                0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 30, 30)            600000    
_________________________________________________________________
lstm_15 (LSTM)               (None, 15)                2760      
_________________________________________________________________
dropout_77 (Dropout)         (None, 15)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 256)               4096      
_________________________________________________________________
dropout_78 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_72 (Dense)             (None, 128)               32896     
__________

In [198]:
model.fit(text_sequence, y_train,validation_split=0.2, epochs=30, verbose=2)

Train on 4816 samples, validate on 1205 samples
Epoch 1/30
 - 3s - loss: 3.0756 - acc: 0.2232 - val_loss: 3.0551 - val_acc: 0.2199
Epoch 2/30
 - 3s - loss: 2.8495 - acc: 0.2377 - val_loss: 2.8860 - val_acc: 0.2266
Epoch 3/30
 - 3s - loss: 2.6505 - acc: 0.2797 - val_loss: 2.6658 - val_acc: 0.2888
Epoch 4/30
 - 3s - loss: 2.4720 - acc: 0.3439 - val_loss: 2.6168 - val_acc: 0.3203
Epoch 5/30


KeyboardInterrupt: 

In [174]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=30)

In [175]:
model.evaluate(test_sequences_matrix, y_test)

2007/2007 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 171us/step


[3.734817006662822, 0.44643746891839026]

In [153]:
model.metrics_names

['loss', 'acc']

In [154]:
y_pred = model.predict(test_sequences_matrix)

In [138]:
y_pred[1]

array([4.3649884e-34, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       4.6759407e-31, 3.6502335e-32, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 7.1507774e-36, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.8421542e-17, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       5.9093618e-17, 0.0000000e+00, 0.0000000e+00, 3.0400352e-24,
       4.2506952e-30, 1.8260527e-13, 6.1317772e-14, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 3.8912346e-30, 8.1327877e-18,
       9.9999893e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.2691032e-10, 0.0000000e+00, 0.0000000e+00], dtype=float32)

In [155]:
test = pd.read_csv(r"C:\Users\elonm\Desktop\timepass\test.csv")
test['question'] = test['question'].progress_apply(lambda txt: str(txt))
test['question'] = test['question'].progress_apply(lambda s: punct_remove(s))
test['question'] = test['question'].progress_apply(lambda s: stemming(s))

In [156]:
test.head()

,ID,question
0,8028,homeopathi autismvaccineantidot
1,8029,what topic appli to atop dermat infant
2,8030,cyst treatment with dian 35
3,8031,I suffer from venou insuffici acrocyanos I fou...
4,8032,I am pregnant for 1 month is that essenti oil ...


In [157]:
test_seq = tokenizer.texts_to_sequences(test['question'])
test_seque = pad_sequences(test_seq,maxlen=30)
test_pred = model.predict(test_seque)

In [158]:
test_pred[1].argmax()

42

6

In [159]:
np.argmin(test_pred[6])

15

In [208]:
def return_int(lst):
    index = np.argmax(lst)
    return index

for i in range(2035):
    test['question'][i] = return_int(test_pred[i])

KeyError: 'question'

In [161]:
test.columns = [['ID', 'Intention']]
test.head()

,ID,Intention
0,8028,22
1,8029,42
2,8030,31
3,8031,32
4,8032,44


In [162]:
test.to_csv(r"C:\Users\elonm\Desktop\timepass\output_test.csv",index=False, sep=";")

In [211]:
# %load C:/Users/elonm/Desktop/timepass/public_accuracy.py
import numpy as np
            
def score_function(y_true, y_pred):
    score = 0
    length1 = y_true.shape[0]
    for i in range(length1):
        if y_pred[i, :] == y_true[i, :]:
            score += 1
    return float(score)/float(length1)

## Using BOW and TF-Idf with Dense layers

In [200]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = TfidfVectorizer(ngram_range=(2,2), max_df=0.80,
                        tokenizer=nltk.word_tokenize,
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=False, 
                        stop_words = 'english')
bag_of_words = count_vectorizer.fit_transform(X_train)
print(bag_of_words.shape)
X_test = count_vectorizer.transform(X_test)
print(X_test.shape)

(6021, 29319)
(2007, 29319)


In [201]:
transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False)
transformer_bag_of_words = transformer.fit_transform(bag_of_words)
X_test_transformer = transformer.transform(X_test)
print (transformer_bag_of_words.shape)
print (X_test_transformer.shape)

(6021, 29319)
(2007, 29319)


In [203]:
model = Sequential()
model.add(Dense(512, input_dim=transformer_bag_of_words.shape[1]))
model.add(Dropout(0.15))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(51, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_crossentropy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 512)               15011840  
_________________________________________________________________
dropout_85 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_86 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_87 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_84 (Dense)             (None, 512)               262656    
__________

In [206]:
model.fit(transformer_bag_of_words, y_train,validation_split=0.2, epochs=30, verbose=1)

Train on 4816 samples, validate on 1205 samples
Epoch 1/30
4352/4816 [==========================>...] - ETA: 46s - loss: 2.9373 - categorical_crossentropy: 2.937 - ETA: 44s - loss: 2.8986 - categorical_crossentropy: 2.898 - ETA: 43s - loss: 2.8878 - categorical_crossentropy: 2.887 - ETA: 43s - loss: 2.8759 - categorical_crossentropy: 2.875 - ETA: 43s - loss: 2.9071 - categorical_crossentropy: 2.907 - ETA: 43s - loss: 2.9882 - categorical_crossentropy: 2.988 - ETA: 42s - loss: 2.9587 - categorical_crossentropy: 2.958 - ETA: 42s - loss: 3.0161 - categorical_crossentropy: 3.016 - ETA: 42s - loss: 3.0176 - categorical_crossentropy: 3.017 - ETA: 42s - loss: 3.0172 - categorical_crossentropy: 3.017 - ETA: 42s - loss: 3.0091 - categorical_crossentropy: 3.009 - ETA: 42s - loss: 3.0045 - categorical_crossentropy: 3.004 - ETA: 41s - loss: 3.0355 - categorical_crossentropy: 3.035 - ETA: 41s - loss: 3.0459 - categorical_crossentropy: 3.045 - ETA: 41s - loss: 3.0564 - categorical_crossentropy: 3.05

4384/4816 [==========================>...] - ETA: 44s - loss: 2.2985 - categorical_crossentropy: 2.298 - ETA: 45s - loss: 2.5325 - categorical_crossentropy: 2.532 - ETA: 43s - loss: 2.6413 - categorical_crossentropy: 2.641 - ETA: 45s - loss: 2.6378 - categorical_crossentropy: 2.637 - ETA: 46s - loss: 2.6848 - categorical_crossentropy: 2.684 - ETA: 45s - loss: 2.6631 - categorical_crossentropy: 2.663 - ETA: 44s - loss: 2.6444 - categorical_crossentropy: 2.644 - ETA: 43s - loss: 2.5721 - categorical_crossentropy: 2.572 - ETA: 43s - loss: 2.5824 - categorical_crossentropy: 2.582 - ETA: 43s - loss: 2.5131 - categorical_crossentropy: 2.513 - ETA: 42s - loss: 2.4933 - categorical_crossentropy: 2.493 - ETA: 42s - loss: 2.5499 - categorical_crossentropy: 2.549 - ETA: 41s - loss: 2.5727 - categorical_crossentropy: 2.572 - ETA: 41s - loss: 2.5497 - categorical_crossentropy: 2.549 - ETA: 40s - loss: 2.5491 - categorical_crossentropy: 2.549 - ETA: 40s - loss: 2.5559 - categorical_crossentropy: 2.5

4816/4816 [==============================] - ETA: 3s - loss: 2.3278 - categorical_crossentropy: 2.32 - ETA: 3s - loss: 2.3238 - categorical_crossentropy: 2.32 - ETA: 3s - loss: 2.3211 - categorical_crossentropy: 2.32 - ETA: 2s - loss: 2.3182 - categorical_crossentropy: 2.31 - ETA: 2s - loss: 2.3153 - categorical_crossentropy: 2.31 - ETA: 2s - loss: 2.3156 - categorical_crossentropy: 2.31 - ETA: 1s - loss: 2.3095 - categorical_crossentropy: 2.30 - ETA: 1s - loss: 2.3071 - categorical_crossentropy: 2.30 - ETA: 1s - loss: 2.3046 - categorical_crossentropy: 2.30 - ETA: 1s - loss: 2.3038 - categorical_crossentropy: 2.30 - ETA: 0s - loss: 2.3005 - categorical_crossentropy: 2.30 - ETA: 0s - loss: 2.2974 - categorical_crossentropy: 2.29 - ETA: 0s - loss: 2.2959 - categorical_crossentropy: 2.29 - 46s 10ms/step - loss: 2.2960 - categorical_crossentropy: 2.2960 - val_loss: 3.1106 - val_categorical_crossentropy: 3.1106
Epoch 3/30
4384/4816 [==========================>...] - ETA: 44s - loss: 1.6477

4384/4816 [==========================>...] - ETA: 42s - loss: 0.6510 - categorical_crossentropy: 0.651 - ETA: 41s - loss: 0.7251 - categorical_crossentropy: 0.725 - ETA: 44s - loss: 0.6714 - categorical_crossentropy: 0.671 - ETA: 43s - loss: 0.6804 - categorical_crossentropy: 0.680 - ETA: 43s - loss: 0.7353 - categorical_crossentropy: 0.735 - ETA: 43s - loss: 0.7384 - categorical_crossentropy: 0.738 - ETA: 43s - loss: 0.7842 - categorical_crossentropy: 0.784 - ETA: 42s - loss: 0.8405 - categorical_crossentropy: 0.840 - ETA: 42s - loss: 0.8504 - categorical_crossentropy: 0.850 - ETA: 42s - loss: 0.8287 - categorical_crossentropy: 0.828 - ETA: 42s - loss: 0.8357 - categorical_crossentropy: 0.835 - ETA: 41s - loss: 0.8603 - categorical_crossentropy: 0.860 - ETA: 41s - loss: 0.8755 - categorical_crossentropy: 0.875 - ETA: 40s - loss: 0.8654 - categorical_crossentropy: 0.865 - ETA: 40s - loss: 0.8851 - categorical_crossentropy: 0.885 - ETA: 40s - loss: 0.8641 - categorical_crossentropy: 0.8

4816/4816 [==============================] - ETA: 3s - loss: 0.8726 - categorical_crossentropy: 0.87 - ETA: 3s - loss: 0.8721 - categorical_crossentropy: 0.87 - ETA: 3s - loss: 0.8719 - categorical_crossentropy: 0.87 - ETA: 3s - loss: 0.8724 - categorical_crossentropy: 0.87 - ETA: 2s - loss: 0.8753 - categorical_crossentropy: 0.87 - ETA: 2s - loss: 0.8788 - categorical_crossentropy: 0.87 - ETA: 2s - loss: 0.8813 - categorical_crossentropy: 0.88 - ETA: 1s - loss: 0.8780 - categorical_crossentropy: 0.87 - ETA: 1s - loss: 0.8771 - categorical_crossentropy: 0.87 - ETA: 1s - loss: 0.8754 - categorical_crossentropy: 0.87 - ETA: 0s - loss: 0.8732 - categorical_crossentropy: 0.87 - ETA: 0s - loss: 0.8729 - categorical_crossentropy: 0.87 - ETA: 0s - loss: 0.8753 - categorical_crossentropy: 0.87 - 48s 10ms/step - loss: 0.8749 - categorical_crossentropy: 0.8749 - val_loss: 4.3884 - val_categorical_crossentropy: 4.3884
Epoch 5/30
4384/4816 [==========================>...] - ETA: 39s - loss: 0.6846

...

4816/4816 [==============================] - ETA: 3s - loss: 0.0811 - categorical_crossentropy: 0.08 - ETA: 3s - loss: 0.0815 - categorical_crossentropy: 0.08 - ETA: 2s - loss: 0.0814 - categorical_crossentropy: 0.08 - ETA: 2s - loss: 0.0819 - categorical_crossentropy: 0.08 - ETA: 2s - loss: 0.0815 - categorical_crossentropy: 0.08 - ETA: 2s - loss: 0.0810 - categorical_crossentropy: 0.08 - ETA: 1s - loss: 0.0805 - categorical_crossentropy: 0.08 - ETA: 1s - loss: 0.0824 - categorical_crossentropy: 0.08 - ETA: 1s - loss: 0.0840 - categorical_crossentropy: 0.08 - ETA: 0s - loss: 0.0838 - categorical_crossentropy: 0.08 - ETA: 0s - loss: 0.0832 - categorical_crossentropy: 0.08 - ETA: 0s - loss: 0.0838 - categorical_crossentropy: 0.08 - ETA: 0s - loss: 0.0834 - categorical_crossentropy: 0.08 - 41s 9ms/step - loss: 0.0836 - categorical_crossentropy: 0.0836 - val_loss: 5.1160 - val_categorical_crossentropy: 5.1160
Epoch 27/30
4384/4816 [==========================>...] - ETA: 35s - loss: 0.0017

4384/4816 [==========================>...] - ETA: 37s - loss: 6.6729e-04 - categorical_crossentropy: 6.6729e-0 - ETA: 38s - loss: 0.0815 - categorical_crossentropy: 0.0815        - ETA: 38s - loss: 0.0548 - categorical_crossentropy: 0.054 - ETA: 38s - loss: 0.0511 - categorical_crossentropy: 0.051 - ETA: 38s - loss: 0.0410 - categorical_crossentropy: 0.041 - ETA: 38s - loss: 0.0512 - categorical_crossentropy: 0.051 - ETA: 38s - loss: 0.0470 - categorical_crossentropy: 0.047 - ETA: 37s - loss: 0.0412 - categorical_crossentropy: 0.041 - ETA: 37s - loss: 0.0413 - categorical_crossentropy: 0.041 - ETA: 37s - loss: 0.0372 - categorical_crossentropy: 0.037 - ETA: 37s - loss: 0.0405 - categorical_crossentropy: 0.040 - ETA: 36s - loss: 0.0376 - categorical_crossentropy: 0.037 - ETA: 36s - loss: 0.0390 - categorical_crossentropy: 0.039 - ETA: 36s - loss: 0.0409 - categorical_crossentropy: 0.040 - ETA: 36s - loss: 0.0462 - categorical_crossentropy: 0.046 - ETA: 35s - loss: 0.0444 - categorical_c

4816/4816 [==============================] - ETA: 3s - loss: 0.0726 - categorical_crossentropy: 0.07 - ETA: 3s - loss: 0.0726 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0728 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0732 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0743 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0738 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0747 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0744 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0770 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0765 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0764 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0761 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0764 - categorical_crossentropy: 0.07 - 41s 9ms/step - loss: 0.0763 - categorical_crossentropy: 0.0763 - val_loss: 4.5685 - val_categorical_crossentropy: 4.5685
Epoch 29/30
4384/4816 [==========================>...] - ETA: 37s - loss: 0.8980

4384/4816 [==========================>...] - ETA: 37s - loss: 7.3677e-04 - categorical_crossentropy: 7.3677e-0 - ETA: 38s - loss: 0.0114 - categorical_crossentropy: 0.0114        - ETA: 38s - loss: 0.0079 - categorical_crossentropy: 0.007 - ETA: 38s - loss: 0.0110 - categorical_crossentropy: 0.011 - ETA: 38s - loss: 0.0180 - categorical_crossentropy: 0.018 - ETA: 38s - loss: 0.0212 - categorical_crossentropy: 0.021 - ETA: 38s - loss: 0.0208 - categorical_crossentropy: 0.020 - ETA: 38s - loss: 0.0204 - categorical_crossentropy: 0.020 - ETA: 38s - loss: 0.0201 - categorical_crossentropy: 0.020 - ETA: 37s - loss: 0.0788 - categorical_crossentropy: 0.078 - ETA: 37s - loss: 0.0771 - categorical_crossentropy: 0.077 - ETA: 37s - loss: 0.0715 - categorical_crossentropy: 0.071 - ETA: 37s - loss: 0.0671 - categorical_crossentropy: 0.067 - ETA: 36s - loss: 0.0637 - categorical_crossentropy: 0.063 - ETA: 36s - loss: 0.0641 - categorical_crossentropy: 0.064 - ETA: 36s - loss: 0.0626 - categorical_c

4816/4816 [==============================] - ETA: 3s - loss: 0.0764 - categorical_crossentropy: 0.07 - ETA: 3s - loss: 0.0759 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0764 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0761 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0756 - categorical_crossentropy: 0.07 - ETA: 2s - loss: 0.0751 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0745 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0754 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0768 - categorical_crossentropy: 0.07 - ETA: 1s - loss: 0.0764 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0759 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0792 - categorical_crossentropy: 0.07 - ETA: 0s - loss: 0.0787 - categorical_crossentropy: 0.07 - 45s 9ms/step - loss: 0.0784 - categorical_crossentropy: 0.0784 - val_loss: 4.6514 - val_categorical_crossentropy: 4.6514


In [207]:
model.evaluate(X_test_transformer, y_test)

2007/2007 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 638us/step


[5.143167510933108, 5.143167510933108]